In [7]:
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()
from openai import OpenAI
import os

In [3]:
uw_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

wsu_info = """
Washington State University, commonly known as WSU, founded in 1890, is a public research university in Pullman, Washington.
With multiple campuses across the state, it is the state's second largest institution of higher education.
WSU is known for its programs in veterinary medicine, agriculture, engineering, architecture, and pharmacy.
"""

seattle_info = """
Seattle, a city on Puget Sound in the Pacific Northwest, is surrounded by water, mountains and evergreen forests, and contains thousands of acres of parkland.
It's home to a large tech industry, with Microsoft and Amazon headquartered in its metropolitan area.
The futuristic Space Needle, a legacy of the 1962 World's Fair, is its most iconic landmark.
"""

starbucks_info = """
Starbucks Corporation is an American multinational chain of coffeehouses and roastery reserves headquartered in Seattle, Washington.
As the world's largest coffeehouse chain, Starbucks is seen to be the main representation of the United States' second wave of coffee culture.
"""

newzealand_info = """
New Zealand is an island country located in the southwestern Pacific Ocean. It comprises two main landmasses—the North Island and the South Island—and over 700 smaller islands.
The country is known for its stunning landscapes, ranging from lush forests and mountains to beaches and lakes. New Zealand has a rich cultural heritage, with influences from 
both the indigenous Māori people and European settlers. The capital city is Wellington, while the largest city is Auckland. New Zealand is also famous for its adventure tourism,
including activities like bungee jumping, skiing, and hiking.
"""

In [5]:
print(seattle_info)


Seattle, a city on Puget Sound in the Pacific Northwest, is surrounded by water, mountains and evergreen forests, and contains thousands of acres of parkland.
It's home to a large tech industry, with Microsoft and Amazon headquartered in its metropolitan area.
The futuristic Space Needle, a legacy of the 1962 World's Fair, is its most iconic landmark.



In [8]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(
    api_key=os.environ.get("OPENAI_API_KEY"),
    model_name="text-embedding-ada-002",
)


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(
    name="Washington", embedding_function=embedding_function
)

In [ ]:
vector_store.add("uw_info", documents=uw_info)
vector_store.add("wsu_info", documents=wsu_info)
vector_store.add("seattle_info", documents=seattle_info)
vector_store.add("starbucks_info", documents=starbucks_info)
vector_store.add("newzealand_info", documents=newzealand_info)

In [24]:
vector_store.count()

5

In [25]:
from trulens.apps.app import instrument
from trulens.core import TruSession

session = TruSession()
session.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [36]:
from openai import OpenAI
oai_client = OpenAI()

In [38]:

class RAG:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(query_texts=query, n_results=4)
        # Flatten the list of lists into a single list
        return [doc for sublist in results["documents"] for doc in sublist]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        if len(context_str) == 0:
            return "Sorry, I couldn't find an answer to your question."

        completion = (
            oai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": f"We have provided context information below. \n"
                        f"---------------------\n"
                        f"{context_str}"
                        f"\n---------------------\n"
                        f"First, say hello and that you're happy to help. \n"
                        f"\n---------------------\n"
                        f"Then, given this information, please answer the question: {query}",
                    }
                ],
            )
            .choices[0]
            .message.content
        )
        if completion:
            return completion
        else:
            return "Did not find an answer."

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query=query)
        completion = self.generate_completion(
            query=query, context_str=context_str
        )
        return completion


rag = RAG()

In [39]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4.1-mini")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve.rets[:])
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets[:] .


In [40]:
from trulens.apps.app import TruApp

tru_rag = TruApp(
    rag,
    app_name="RAG",
    app_version="base",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

instrumenting <class '__main__.RAG'> for base <class '__main__.RAG'>
	instrumenting retrieve
	instrumenting generate_completion
	instrumenting query


In [41]:
with tru_rag as recording:
    rag.query(
        "What wave of coffee culture is Starbucks seen to represent in the United States?"
    )
    rag.query(
        "What wave of coffee culture is Starbucks seen to represent in the New Zealand?"
    )
    rag.query("Does Washington State have Starbucks on campus?")

In [42]:
session.get_leaderboard()

Answer Relevance  Context Relevance  Groundedness  \
app_name app_version                                                      
RAG      base                 1.000000           0.166667      0.733333   
         filtered             0.333333           1.000000      1.000000   

                       latency  total_cost  
app_name app_version                        
RAG      base         1.507925    0.000621  
         filtered     2.365966    0.000078

In [44]:
from trulens.core.guardrails.base import context_filter

guardrail_provider = OpenAI(model_engine="gpt-4.1-nano")

# note: feedback function used for guardrail must only return a score, not also reasons
f_context_relevance_score = Feedback(
    guardrail_provider.context_relevance, name="Context Relevance"
)


class FilteredRAG(RAG):
    @instrument
    @context_filter(
        feedback=f_context_relevance_score,
        threshold=0.75,
        keyword_for_prompt="query",
    )
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(query_texts=query, n_results=4)
        if "documents" in results and results["documents"]:
            return [doc for sublist in results["documents"] for doc in sublist]
        else:
            return []


filtered_rag = FilteredRAG()

In [45]:
from trulens.apps.custom import TruCustomApp

filtered_tru_rag = TruCustomApp(
    filtered_rag,
    app_name="RAG",
    app_version="filtered",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

with filtered_tru_rag as recording:
    filtered_rag.query(
        query="What wave of coffee culture is Starbucks seen to represent in the United States?"
    )
    filtered_rag.query(
        "What wave of coffee culture is Starbucks seen to represent in the New Zealand?"
    )
    filtered_rag.query("Does Washington State have Starbucks on campus?")

C:\Users\Al-Amin\AppData\Local\Temp\ipykernel_57716\3098305819.py:3: DeprecationWarning: TruCustomApp is being deprecated in the next major version; use TruApp instead.
  filtered_tru_rag = TruCustomApp(


instrumenting <class '__main__.FilteredRAG'> for base <class '__main__.FilteredRAG'>
	instrumenting retrieve
	instrumenting generate_completion
	instrumenting query
	instrumenting retrieve
instrumenting <class '__main__.FilteredRAG'> for base <class '__main__.RAG'>
	instrumenting retrieve
	instrumenting generate_completion
	instrumenting query
	instrumenting retrieve


D:\Al-Amin\llama_parse_al\env\lib\site-packages\trulens\core\feedback\feedback.py:1023: UserWarning: Feedback function Context Relevance with aggregation <function mean at 0x0000025AAD0619D0> had no inputs.
  warnings.warn(
D:\Al-Amin\llama_parse_al\env\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Al-Amin\llama_parse_al\env\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [46]:
session.get_leaderboard()

Answer Relevance  Context Relevance  Groundedness  \
app_name app_version                                                      
RAG      base                 1.000000           0.166667      0.777778   
         filtered             0.333333           1.000000      1.000000   

                       latency  total_cost  
app_name app_version                        
RAG      base         1.507925    0.000621  
         filtered     2.200799    0.000078

In [47]:
from trulens.dashboard import run_dashboard

run_dashboard(session)

Starting dashboard ...
Dashboard already running at path:   Local URL: http://localhost:56181



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [53]:
from openai import OpenAI
openai = OpenAI()

In [56]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4.1-mini")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve.rets[:])
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets[:] .


In [57]:
from trulens.apps.app import TruApp

tru_rag = TruApp(
    rag,
    app_name="RAG",
    app_version="base",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

instrumenting <class '__main__.RAG'> for base <class '__main__.RAG'>
	instrumenting retrieve
	instrumenting generate_completion
	instrumenting query


In [72]:
#Deveoping the app
from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core import load_index_from_storage
import os


In [73]:
def build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            [document], service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index

In [114]:
import os
import logging
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
    Document
)
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import load_index_from_storage

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def build_sentence_window_index(
    document: Document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
):
    """
    Build or load a sentence window index for a given document.

    Args:
        document (Document): The input document to index.
        llm: The language model to use (e.g., OpenAI).
        embed_model (str): The embedding model (default: BAAI/bge-small-en-v1.5).
        save_dir (str): Directory to persist the index (default: sentence_index).

    Returns:
        VectorStoreIndex: The built or loaded sentence window index.
    """
    try:
        # Configure global settings
        Settings.llm = llm
        Settings.embed_model = embed_model

        # Create the sentence window node parser with default settings
        node_parser = SentenceWindowNodeParser.from_defaults(
            window_size=3,
            window_metadata_key="window",
            original_text_metadata_key="original_text"
        )
        Settings.node_parser = node_parser

        # Parse document into nodes
        nodes = node_parser.get_nodes_from_documents([document])

        if not os.path.exists(save_dir):
            # Build new index
            logger.info(f"Creating new index at {save_dir}")
            sentence_index = VectorStoreIndex(nodes)
            sentence_index.storage_context.persist(persist_dir=save_dir)
        else:
            # Load existing index
            logger.info(f"Loading index from {save_dir}")
            storage_context = StorageContext.from_defaults(persist_dir=save_dir)
            sentence_index = load_index_from_storage(storage_context)

        return sentence_index

    except Exception as e:
        logger.error(f"Error building/loading sentence window index: {e}")
        raise

In [122]:
def get_sentence_window_query_engine(
    sentence_index,
    similarity_top_k=6,
    rerank_top_n=2,
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [81]:
from llama_index.core.node_parser import HierarchicalNodeParser

from llama_index.core.node_parser import get_leaf_nodes
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [83]:
def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index

In [84]:
def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=2,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [85]:
#Lesson 2: RAG Triad of metrics

In [103]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Document
from llama_index.llms.openai import OpenAI

In [90]:

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [104]:
document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [107]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [99]:
print(type(documents))
print(len(documents))
type(documents[0])

<class 'list'>
41


llama_index.core.schema.Document

In [116]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
OpenAI_Embedding = OpenAIEmbedding(model="text-embedding-ada-002")

In [118]:
import os
import logging
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
    Document
)
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import load_index_from_storage

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def build_sentence_window_index(
    document: Document,
    llm,
    embed_model=OpenAI_Embedding,
    save_dir="sentence_index"
):
    """
    Build or load a sentence window index for a given document.

    Args:
        document (Document): The input document to index.
        llm: The language model to use (e.g., OpenAI).
        embed_model (str): The embedding model (default: BAAI/bge-small-en-v1.5).
        save_dir (str): Directory to persist the index (default: sentence_index).

    Returns:
        VectorStoreIndex: The built or loaded sentence window index.
    """
    try:
        # Configure global settings
        Settings.llm = llm
        Settings.embed_model = embed_model

        # Create the sentence window node parser with default settings
        node_parser = SentenceWindowNodeParser.from_defaults(
            window_size=3,
            window_metadata_key="window",
            original_text_metadata_key="original_text"
        )
        Settings.node_parser = node_parser

        # Parse document into nodes
        nodes = node_parser.get_nodes_from_documents([document])

        if not os.path.exists(save_dir):
            # Build new index
            logger.info(f"Creating new index at {save_dir}")
            sentence_index = VectorStoreIndex(nodes)
            sentence_index.storage_context.persist(persist_dir=save_dir)
        else:
            # Load existing index
            logger.info(f"Loading index from {save_dir}")
            storage_context = StorageContext.from_defaults(persist_dir=save_dir)
            sentence_index = load_index_from_storage(storage_context)

        return sentence_index

    except Exception as e:
        logger.error(f"Error building/loading sentence window index: {e}")
        raise

In [119]:
sentence_index = build_sentence_window_index(
    document,
    llm,
    save_dir="sentence_index"
)

In [123]:
sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

D:\Al-Amin\llama_parse_al\env\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Al-Amin\.cache\huggingface\hub\models--BAAI--bge-reranker-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [124]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
output.response

'You create your AI portfolio by showcasing a progression of projects that demonstrate your skills in AI.'

In [125]:
output

Response(response='You create your AI portfolio by showcasing a progression of projects that demonstrate your skills in AI.', source_nodes=[NodeWithScore(node=TextNode(id_='7c6cdb2c-571f-413e-bf87-e5c21fed67b9', embedding=None, metadata={'window': 'Chapter 4: Scoping Successful AI Projects.\n Chapter 5: Finding Projects that Complement \nYour Career Goals.\n Chapter 6: Building a Portfolio of Projects that \nShows Skill Progression.\n Chapter 7: A Simple Framework for Starting Your AI \nJob Search.\n Chapter 8: Using Informational Interviews to Find \nthe Right Job.\n Chapter 9: Finding the Right AI Job for You.\n Chapter 10: Keys to Building a Career in AI.\n', 'original_text': 'Chapter 7: A Simple Framework for Starting Your AI \nJob Search.\n'}, excluded_embed_metadata_keys=['window', 'original_text'], excluded_llm_metadata_keys=['window', 'original_text'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7b812dd1-3a21-439d-a101-2317c4dee3f0', node_type='4', m

In [ ]:
f_groundedness, f_answer_relevance, f_context_relevance

In [128]:
import numpy as np
from trulens.apps.llamaindex import TruLlama
from trulens.core import Feedback
from trulens.providers.openai import OpenAI

# Initialize provider class
provider = OpenAI(model_engine="gpt-4.1-mini")

# select context to be used in feedback. the location of context is app specific.

context = TruLlama.select_context(sentence_window_engine)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())  # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


In [129]:
tru_query_engine_recorder = TruLlama(
    sentence_window_engine,
    app_name="LlamaIndex_App",
    app_version="base",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'abc.ABC'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'

In [130]:
# or as context manager
with tru_query_engine_recorder as recording:
    sentence_window_engine.query("What did the author do growing up?")

In [131]:
run_dashboard(session)

Starting dashboard ...
Dashboard already running at path:   Local URL: http://localhost:56181



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>